In [309]:
!pip install kagglehub

In [310]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import kagglehub
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import load_iris
import numpy as np

In [311]:
# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")

Используется устройство: cpu
Path to dataset files: C:\Users\Manya\.cache\kagglehub\datasets\uciml\iris\versions\2


In [312]:
iris = load_iris()
X = iris.data  
y = iris.target

In [313]:
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Масштабирование признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование в тензоры PyTorch
X_train_tensor = torch.FloatTensor(X_train).to(device)
X_test_tensor = torch.FloatTensor(X_test).to(device)
y_train_tensor = torch.LongTensor(y_train).to(device)
y_test_tensor = torch.LongTensor(y_test).to(device)

In [314]:
print(X_train.shape, y_train.shape)

(120, 4) (120,)


In [315]:
class IrisDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
    def __len__(self):
        return len(self.features)

train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)

In [316]:
class IrisNet(nn.Module):
    def __init__(self, num_inputs=4, num_outputs=3):
        super().__init__()
        self.layers = nn.Sequential(
            # Input layer
            nn.Linear(num_inputs, 16),
            nn.ReLU(),
            nn.Dropout(0.2),
            # Hidden layer
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Dropout(0.2),
            # Output layer
            nn.Linear(8, num_outputs)
        )
    
    def forward(self, x):
        return self.layers(x)

In [317]:
torch.manual_seed(42)
model = IrisNet()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) #optim.Adam точность выше

print(f"Количество параметров: {sum(p.numel() for p in model.parameters())}")

Количество параметров: 243


In [318]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=8, shuffle=True, num_workers=0
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=8, shuffle=False, num_workers=0
)
for (idx, (x, y)) in enumerate(train_loader):
    print(f'Batch: #{idx}, \nx: {x}, \ny: {y}')

Batch: #0, 
x: tensor([[ 1.0250, -0.1082,  0.8120,  1.4418],
        [ 0.1891, -0.7799,  0.7552,  0.5201],
        [ 0.1891, -0.3321,  0.4145,  0.3884],
        [-0.4080,  1.0112, -1.4025, -1.3233],
        [ 0.7861, -0.5560,  0.4713,  0.3884],
        [ 0.6667, -0.3321,  0.3009,  0.1251],
        [-1.2439,  0.7873, -1.0618, -1.3233],
        [-0.0498, -0.7799,  0.0738, -0.0066]]), 
y: tensor([2, 1, 1, 0, 1, 1, 0, 1])
Batch: #1, 
x: tensor([[-0.8857,  1.6829, -1.0618, -1.0599],
        [-0.8857,  1.0112, -1.3457, -1.1916],
        [-0.8857,  1.4590, -1.2889, -1.0599],
        [-0.7662,  2.3546, -1.2889, -1.4549],
        [ 1.6221, -0.1082,  1.1527,  0.5201],
        [-1.7216, -0.3321, -1.3457, -1.3233],
        [-1.1245,  1.2351, -1.3457, -1.4549],
        [-1.2439, -0.1082, -1.3457, -1.1916]]), 
y: tensor([0, 0, 0, 0, 2, 0, 0, 0])
Batch: #2, 
x: tensor([[ 1.1444,  0.3396,  1.2094,  1.4418],
        [ 0.7861,  0.3396,  0.7552,  1.0468],
        [ 1.0250,  0.1157,  0.5281,  0.3884],
   

In [319]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()

    for (idx, (x, y)) in enumerate(train_loader):
        model_result = model(x)
        loss = F.cross_entropy(model_result, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Batch: #{epoch}/{idx}, loss: {loss:.2f}')

Batch: #0/0, loss: 1.11
Batch: #0/1, loss: 1.05
Batch: #0/2, loss: 1.05
Batch: #0/3, loss: 1.01
Batch: #0/4, loss: 1.13
Batch: #0/5, loss: 1.05
Batch: #0/6, loss: 1.18
Batch: #0/7, loss: 1.16
Batch: #0/8, loss: 1.07
Batch: #0/9, loss: 1.07
Batch: #0/10, loss: 1.06
Batch: #0/11, loss: 1.07
Batch: #0/12, loss: 1.18
Batch: #0/13, loss: 1.14
Batch: #0/14, loss: 1.08
Batch: #1/0, loss: 1.08
Batch: #1/1, loss: 1.02
Batch: #1/2, loss: 1.08
Batch: #1/3, loss: 1.14
Batch: #1/4, loss: 1.11
Batch: #1/5, loss: 1.11
Batch: #1/6, loss: 1.14
Batch: #1/7, loss: 1.06
Batch: #1/8, loss: 1.12
Batch: #1/9, loss: 1.07
Batch: #1/10, loss: 1.10
Batch: #1/11, loss: 1.12
Batch: #1/12, loss: 1.00
Batch: #1/13, loss: 1.11
Batch: #1/14, loss: 1.00
Batch: #2/0, loss: 1.11
Batch: #2/1, loss: 1.01
Batch: #2/2, loss: 1.08
Batch: #2/3, loss: 1.06
Batch: #2/4, loss: 1.07
Batch: #2/5, loss: 1.13
Batch: #2/6, loss: 1.15
Batch: #2/7, loss: 1.03
Batch: #2/8, loss: 1.07
Batch: #2/9, loss: 1.05
Batch: #2/10, loss: 1.05
Batch

In [320]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

accuracy = 100 * correct / total
print(f'\nТочность на тестовой выборке: {accuracy:.2f}%')



Точность на тестовой выборке: 90.00%


In [321]:
model.eval()
for (idx, (x, y)) in enumerate(test_loader):
    with torch.no_grad():
        outputs = torch.argmax(torch.softmax(model(x), dim=1), dim=1)

        print(f'Batch: #{idx}, output: {outputs}, y: {y}')

Batch: #0, output: tensor([0, 2, 1, 1, 0, 2, 0, 0]), y: tensor([0, 2, 1, 1, 0, 1, 0, 0])
Batch: #1, output: tensor([2, 1, 2, 2, 2, 1, 0, 0]), y: tensor([2, 1, 2, 2, 2, 1, 0, 0])
Batch: #2, output: tensor([0, 1, 1, 2, 0, 2, 1, 1]), y: tensor([0, 1, 1, 2, 0, 2, 1, 2])
Batch: #3, output: tensor([2, 2, 1, 0, 2, 0]), y: tensor([2, 1, 1, 0, 2, 0])


In [322]:
print("\nПример предсказания")
model.eval()
with torch.no_grad():
    sample = test_dataset[0]
    x_sample = sample[0].unsqueeze(0)
    output = model(x_sample)
    probabilities = F.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1)
    
    print(f"Входные данные: {x_sample.squeeze().numpy()}")
    print(f"Вероятности: {probabilities.squeeze().numpy()}")
    print(f"Предсказанный класс: {predicted_class.item()}")
    print(f"Истинный класс: {sample[1].item()}")
    print(f"Название вида: {iris.target_names[predicted_class.item()]}")


Пример предсказания
Входные данные: [-1.7215677  -0.10821272 -1.4025038  -1.3232756 ]
Вероятности: [9.9090827e-01 8.3714798e-03 7.2028802e-04]
Предсказанный класс: 0
Истинный класс: 0
Название вида: setosa
